In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b-deduped")

def count_hf_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

In [2]:
count_hf_tokens('100')

1

In [2]:
import random
import os
import csv

In [3]:
total_examples = 6000
train_size = int(0.9 * total_examples)  # 5400
test_size = total_examples - train_size  # 600

In [4]:
def is_valid_number(num, base):
    for digit in num:
        if not digit.isdigit() or int(digit) >= base:
            return False
    return True

def base_add(num1, num2, base):
    if not (is_valid_number(num1, base) and is_valid_number(num2, base)):
        return False
    
    max_len = max(len(num1), len(num2))
    num1 = num1.zfill(max_len)
    num2 = num2.zfill(max_len)
    
    carry = 0
    result = []
    
    for i in range(max_len - 1, -1, -1):
        digit_sum = int(num1[i]) + int(num2[i]) + carry
        carry = digit_sum // base
        remainder = digit_sum % base  
        
        result.append(str(remainder))
    if carry > 0:
        result.append(str(carry))
    
    result.reverse()
    return ''.join(result)

num1 = "5"
num2 = "4"
print(f"{num1}_9 + {num2}_9 = {base_add(num1, num2, 9)}_9")


5_9 + 4_9 = 10_9


In [5]:
def base_add_twisted_carry(num1, num2, base):
    max_len = max(len(num1), len(num2))
    num1 = num1.zfill(max_len)
    num2 = num2.zfill(max_len)

    carry = 0
    result = []

    for i in range(max_len - 1, -1, -1):
        digit_sum = int(num1[i]) + int(num2[i]) + carry
        carry = 2 * (digit_sum // base)  
        remainder = digit_sum % base
        result.append(str(remainder))

    if carry > 0:
        result.append(str(carry))
    
    result.reverse()
    return ''.join(result)

num1 = "5"
num2 = "4"
print(f"{num1}_9 + {num2}_9 = {base_add(num1, num2, 10)}_9")

5_9 + 4_9 = 9_9


In [6]:
prefix_prompt = 'Solve the following and respond with only the final answer:'

In [7]:
correct_prompt_templates = lambda n1, n2: f'{n1}+{n2}='

In [18]:
import random

all_combos = [(a, b) for a in range(100, 1000) for b in range(100, 1000)]
random.shuffle(all_combos)

results_train = [['clean', 'corrupted', 'answer']]
results_test = [['clean', 'corrupted', 'answer']]

train_count = 0
test_count = 0


for a, b in all_combos:
    if (train_count >= train_size) and (test_count  >= test_size):
        break
    if not int(base_add_twisted_carry(str(a),str(b),9)):
        continue
    
    answer = int(base_add_twisted_carry(str(a),str(b),8))

    if answer <= 0:
        continue 
    if count_hf_tokens(str(answer)) != 1:
        continue

    c = random.randint(100, 1000)
    clean_prompt = prefix_prompt + ' ' + correct_prompt_templates(a, b)
    corrupted_prompt = prefix_prompt + ' ' + correct_prompt_templates(a, c)
    while c == b or (count_hf_tokens(clean_prompt) != count_hf_tokens(corrupted_prompt)):
        c = random.randint(100, 1000)
        corrupted_prompt = prefix_prompt + ' ' + correct_prompt_templates(a, c)
    
    print(count_hf_tokens(clean_prompt),count_hf_tokens(corrupted_prompt))
    if train_count < train_size:
        results_train.append([clean_prompt, corrupted_prompt, answer])
        train_count += 1

    elif test_count < test_size:
        results_test.append([clean_prompt, corrupted_prompt, answer])
        test_count += 1

    print(train_count,test_count)

15 15
1 0
15 15
2 0
15 15
3 0
15 15
4 0
15 15
5 0
15 15
6 0
16 16
7 0
15 15
8 0
15 15
9 0
15 15
10 0
15 15
11 0
15 15
12 0
15 15
13 0
15 15
14 0
15 15
15 0
15 15
16 0
15 15
17 0
15 15
18 0
15 15
19 0
15 15
20 0
15 15
21 0
15 15
22 0
15 15
23 0
15 15
24 0
15 15
25 0
15 15
26 0
15 15
27 0
15 15
28 0
15 15
29 0
15 15
30 0
15 15
31 0
15 15
32 0
15 15
33 0
15 15
34 0
16 16
35 0
15 15
36 0
15 15
37 0
15 15
38 0
15 15
39 0
15 15
40 0
15 15
41 0
15 15
42 0
15 15
43 0
15 15
44 0
15 15
45 0
15 15
46 0
15 15
47 0
15 15
48 0
15 15
49 0
15 15
50 0
15 15
51 0
15 15
52 0
15 15
53 0
15 15
54 0
15 15
55 0
15 15
56 0
15 15
57 0
15 15
58 0
15 15
59 0
15 15
60 0
15 15
61 0
15 15
62 0
15 15
63 0
15 15
64 0
15 15
65 0
15 15
66 0
15 15
67 0
15 15
68 0
15 15
69 0
15 15
70 0
15 15
71 0
15 15
72 0
15 15
73 0
15 15
74 0
15 15
75 0
15 15
76 0
15 15
77 0
15 15
78 0
15 15
79 0
15 15
80 0
15 15
81 0
15 15
82 0
15 15
83 0
15 15
84 0
15 15
85 0
15 15
86 0
15 15
87 0
15 15
88 0
15 15
89 0
15 15
90 0
15 15
91 0
15 15
92

In [19]:
output_directory = 'arithmetic-bases/Base9Double/datasets_csv/'

os.makedirs(output_directory, exist_ok=True)

filename_train = os.path.join(output_directory, "train.csv")
filename_test = os.path.join(output_directory, "validation.csv")

with open(filename_train, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(results_train)

with open(filename_test, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(results_test)

In [24]:
function = lambda a, b: a-b
correct_prompt_templates = lambda n1, n2: f'{n1}+{n2}='

In [27]:
function = lambda a, b: a - b if a > b else a + b
correct_prompt_templates = lambda n1, n2: f'{n1}◇{n2}='

In [28]:
all_combos = [(a, b) for a in range(100, 1000) for b in range(100, 1000)]
random.shuffle(all_combos)

results_train = [['clean', 'corrupted', 'answer']]
results_test = [['clean', 'corrupted', 'answer']]

train_count = 0
test_count = 0


for a, b in all_combos:
    if train_count >= train_size and test_count >= test_size:
        break 

    answer = function(a, b)
    if answer <= 0:
        continue

    c = random.randint(100, 1000)
    while c == b:
        c = random.randint(100, 1000)

    if count_hf_tokens(str(answer)) != 1:
        print(answer)
        continue
    

    prompt = correct_prompt_templates(a, b)
    corrupted_prompt = correct_prompt_templates(a, c)

    clean_prompt = prefix_prompt + ' ' + prompt
    corrupted_prompt = prefix_prompt + ' ' + corrupted_prompt

    if train_count < train_size:
        results_train.append([clean_prompt, corrupted_prompt, answer])
        train_count += 1
    elif test_count < test_size:
        results_test.append([clean_prompt, corrupted_prompt, answer])
        test_count += 1


1076
1233
1903
1037
1577
1277
1349
1029
1121
1781
1803
1614
976
742
1477
1441
1334
1549
1633
1042
1906
1255
1207
1187
1145
1537
1065
1722
749
1451
1402
1124
849
928
719
1030
1136
1255
1812
1243
1010
914
1048
815
766
1498
1807
939
1191
977
938
1273
1644
1141
1070
1640
1310
1198
1670
1527
1506
1587
826
986
1003
731
824
1208
1483
868
1646
836
743
835
899
1334
1381
845
1085
1533
1465
1187
1461
1215
1374
1346
1266
1742
1166
1612
1468
1056
1395
1098
1149
799
1178
1432
1151
1465
1333
1785
1769
1245
829
1382
982
1573
669
1180
1293
1312
1070
1068
1084
1058
1431
1816
1299
1445
961
1236
1219
1166
1235
1148
1078
982
1514
1536
1195
1233
1506
1012
928
1147
977
1118
1313
1083
1450
1320
1203
1136
738
1164
788
742
868
1015
582
1857
1048
1305
1253
1031
1765
1462
788
1453
1096
1014
1396
1262
1089
968
1099
1026
1317
1221
1093
1067
1232
1039
1595
1367
1192
1396
1192
1448
1729
815
1881
1289
1071
1482
1718
1497
1061
1407
934
1378
1068
1185
1414
1071
1470
1599
1040
1250
1425
1790
1085
1697
1075
1031
1372
1516

In [29]:
output_directory = 'three-digit-arithmetic/CondAddSub/datasets_csv/'

os.makedirs(output_directory, exist_ok=True)

filename_train = os.path.join(output_directory, "train.csv")
filename_test = os.path.join(output_directory, "validation.csv")

with open(filename_train, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(results_train)

with open(filename_test, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(results_test)